In [2]:
import sys
sys.path.append("./")
from utils import pkl_load, pkl_dump, text_load

In [3]:
import pandas as pd
from collections import defaultdict
from copy import deepcopy

In [4]:
from concurrent.futures import ProcessPoolExecutor

In [ ]:
"""
Readme

1. we will create a mapping between ndc and rxcui
2. we will create a mapping between rxcui and its ingradient level rxcui
3. we will create a mapping between unused rxcui and its current version
"""

In [66]:
df2 = pd.read_csv("../raw_data/RxNorm/rrf/RXNCONSO.RRF", dtype=str, header=None, sep="|")
df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,3,ENG,NaN,NaN,NaN,NaN,NaN,8717795,NaN,58488005,NaN,SNOMEDCT_US,PT,58488005,"1,4-alpha-Glucan branching enzyme",NaN,N,NaN,NaN
1,3,ENG,NaN,NaN,NaN,NaN,NaN,8717796,NaN,58488005,NaN,SNOMEDCT_US,FN,58488005,"1,4-alpha-Glucan branching enzyme (substance)",NaN,N,NaN,NaN
2,3,ENG,NaN,NaN,NaN,NaN,NaN,8717808,NaN,58488005,NaN,SNOMEDCT_US,SY,58488005,"Amylo-(1,4,6)-transglycosylase",NaN,N,NaN,NaN
3,3,ENG,NaN,NaN,NaN,NaN,NaN,8718164,NaN,58488005,NaN,SNOMEDCT_US,SY,58488005,Branching enzyme,NaN,N,NaN,NaN
4,19,ENG,NaN,NaN,NaN,NaN,NaN,10794494,NaN,112116001,NaN,SNOMEDCT_US,SY,112116001,17-hydrocorticosteroid,NaN,N,NaN,NaN


In [80]:
valid_copts = set(df2[0])

In [106]:
rxcui2name = defaultdict(set)

def f2(x):
    rxcui = x[0]
    name = x[14]
    rxcui2name[rxcui].add(name)

xx = df2.apply(f2, axis=1)

In [107]:
pkl_dump(rxcui2name, "../resources/rxcui2name.pkl")

In [75]:
# outdated concepts
df1 = pd.read_csv("../raw_data/RxNorm/rrf/RXNCUI.RRF", dtype=str, header=None, sep="|")
df1.head()

,0,1,2,3,4,5
0,91669,RXNORM_17AA_170905F,RXNORM_18AB_181203F,1,91669,NaN
1,91714,RXNORM_10AA_100802F,RXNORM_10AA_100802F,1,1000992,NaN
2,91811,RXNORM_04AC_050210F,RXNORM_09AA_090706F,1,857082,NaN
3,91812,RXNORM_04AC_050210F,RXNORM_09AA_090706F,1,857087,NaN
4,91919,RXNORM_04AC_050210F,RXNORM_09AA_090504F,1,849580,NaN


In [82]:
temp = defaultdict(set)

def f1(x):
    old = x[0]
    cur = x[4]
    if old == cur and cur not in valid_copts:
        return
    temp[old].add(cur)

xx = df1.apply(f1, axis=1)

In [84]:
s = set(df1[0])
ntemp = deepcopy(temp)

flag = True
while flag:
    flag = False
    for k, v in temp.items():
        for each in v:
            if each in s:
                if each not in temp:
                    continue
                flag = True
                nv = temp[each]
                for e in nv:
                    ntemp[k].add(e)
                ntemp[k].remove(each)
    temp = deepcopy(ntemp)

In [90]:
old2cur = dict()

for k, v in ntemp.items():
    if len(v) > 0:
        old2cur[k] = v

pkl_dump(old2cur, "../resources/rxcui_remap.pkl")

In [154]:
df3 = pd.read_csv("../raw_data/RxNorm/rrf/RXNREL.RRF", dtype=str, header=None, sep="|")
df3.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,NaN,5,AUI,SY,NaN,6,AUI,permuted_term_of,155592245,NaN,MSH,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,5,SDUI,SIB,NaN,104746,SDUI,NaN,154524204,NaN,MSH,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,5,SDUI,RN,NaN,609702,SDUI,mapped_to,154691227,NaN,MSH,NaN,1,NaN,NaN,NaN,NaN
3,NaN,5,AUI,SY,NaN,2666961,AUI,sort_version_of,155371534,NaN,MSH,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,5,AUI,SY,NaN,2681015,AUI,entry_version_of,155054914,NaN,MSH,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "../raw_data/RxNorm_Drug_Relationships.png")

In [116]:
ingredients = set(df2[(df2[12]=='IN') & (df2[11]=='RXNORM')][0])

In [197]:
ingrad_map = dict()

for ingrd in ingredients:
    ingrad_map[ingrd] = set()
    
    s = list(set(df3[df3[0]==ingrd][4]))
    exist = set(deepcopy(s))
    
    while len(s) > 0:
        sub_ingrad = s.pop()
        ingrad_map[ingrd].add(sub_ingrad)        
#         sub_df = df3[(df3[0]==sub_ingrad) & (df3[7]=='isa') & (df3[7]=='constitutes') & (df3[7]=='tradename_of')]
        sub_df = df3[(df3[0]==sub_ingrad) & ((df3[7]=='isa') | (df3[7]=='tradename_of') | (df3[7]=='constitutes') | (df3[7]=='has_ingredient'))]
        for e in set(sub_df[4]):
            if e not in ingredients and e not in exist:
                exist.add(e)
                s.append(e)










  0%|          | 0/12969 [00:00<?, ?it/s]








  0%|          | 1/12969 [00:00<28:57,  7.46it/s]








  0%|          | 2/12969 [00:00<28:34,  7.56it/s]








  0%|          | 3/12969 [00:00<28:42,  7.53it/s]








  0%|          | 4/12969 [00:15<16:30:12,  4.58s/it]








  0%|          | 5/12969 [00:29<26:44:51,  7.43s/it]








  0%|          | 6/12969 [01:41<96:20:49, 26.76s/it]








  0%|          | 7/12969 [01:41<67:34:51, 18.77s/it]








  0%|          | 8/12969 [01:41<47:26:41, 13.18s/it]








  0%|          | 9/12969 [01:59<52:23:52, 14.55s/it]








  0%|          | 10/12969 [02:15<53:52:22, 14.97s/it]








  0%|          | 11/12969 [02:15<37:51:00, 10.52s/it]








  0%|          | 12/12969 [02:29<41:58:20, 11.66s/it]








  0%|          | 13/12969 [02:30<30:26:24,  8.46s/it]








  0%|          | 14/12969 [02:30<21:27:02,  5.96s/it]








  0%|          | 15/12969 [02:30<15:09:08,  4.21s/it]
KeyboardInterrupt



In [200]:
def helper(ingrd):
    ingrad_map = dict()
    ingrad_map[ingrd] = set()
    
    s = list(set(df3[df3[0]==ingrd][4]))
    exist = set(deepcopy(s))
    
    while len(s) > 0:
        sub_ingrad = s.pop()
        ingrad_map[ingrd].add(sub_ingrad)        
#         sub_df = df3[(df3[0]==sub_ingrad) & (df3[7]=='isa') & (df3[7]=='constitutes') & (df3[7]=='tradename_of')]
        sub_df = df3[(df3[0]==sub_ingrad) & ((df3[7]=='isa') | (df3[7]=='tradename_of') | (df3[7]=='constitutes') | (df3[7]=='has_ingredient'))]
        for e in set(sub_df[4]):
            if e not in ingredients and e not in exist:
                exist.add(e)
                s.append(e)
    return ingrad_map

In [203]:
final_ingra_map = dict()

with ProcessPoolExecutor(max_workers=8) as exe:
    for each in exe.map(helper, ingredients):
        final_ingra_map.update(each)

KeyboardInterrupt: 

Process ForkProcess-3:
Process ForkProcess-7:
Process ForkProcess-12:
Process ForkProcess-9:
Process ForkProcess-4:
Process ForkProcess-2:
Process ForkProcess-11:
Process ForkProcess-1:
Process ForkProcess-10:
Process ForkProcess-6:
Traceback (most recent call last):
Traceback (most recent call last):


In [205]:
rxcui2ingre = defaultdict(set)
for k, v in final_ingra_map.items():
    for each in v:
        rxcui2ingre[each] = k
pkl_dump(rxcui2ingre, "../resources/rxcui2ingredient.pkl")

In [6]:
#ndc to rxcui
df4 = pd.read_csv("../raw_data/RxNorm/rrf/RXNSAT.RRF", dtype=str, header=None, sep="|")
df4 = df4[df4[8]=='NDC']
df4.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,3,NaN,NaN,8717795,AUI,58488005,NaN,NaN,UMLSAUI,RXNORM,A27769867,NaN,NaN,NaN
1,3,NaN,NaN,8717795,AUI,58488005,NaN,NaN,UMLSCUI,RXNORM,C0000052,NaN,NaN,NaN
2,3,NaN,NaN,8717796,AUI,58488005,NaN,NaN,UMLSAUI,RXNORM,A27780666,NaN,NaN,NaN
3,3,NaN,NaN,8717796,AUI,58488005,NaN,NaN,UMLSCUI,RXNORM,C0000052,NaN,NaN,NaN
4,3,NaN,NaN,8717808,AUI,58488005,NaN,NaN,UMLSAUI,RXNORM,A27766654,NaN,NaN,NaN


In [11]:
ndc2rxcui = dict()

def f4(x):
    ndc = x[10]
    rxcui = x[0]
    ndc2rxcui[ndc] = rxcui
    
xx = df4.apply(f4, axis=1)

In [12]:
len(ndc2rxcui)

1403439

In [13]:
pkl_dump(ndc2rxcui, "../resources/ndc2rxcui.pkl")